In [8]:
import numpy as np

from jitr import reactions, rmatrix
from jitr.utils.kinematics import classical_kinematics


def potential_scalar(r, depth, mass):
    return depth * np.exp(-r / mass)


def potential_2level(r, depth, mass, coupling):
    diag = potential_scalar(r, depth, mass)
    off_diag = potential_scalar(r, coupling, mass)
    return np.array(
        [[diag, off_diag], [off_diag, diag]],
    )


nbasis = 40
solver = rmatrix.Solver(nbasis)

nchannels = 2
sys = reactions.ProjectileTargetSystem(
    5 * np.pi * np.ones(nchannels),
    np.arange(0, nchannels, dtype=np.int64),
    mass_target=44657,
    mass_projectile=938.3,
    Ztarget=20,
    Zproj=1,
    nchannels=nchannels,
)

Elab = 42.1
mu, Ecm, k, eta = classical_kinematics(
    sys.mass_target, sys.mass_projectile, Elab, sys.Zproj * sys.Ztarget
)

# for solving the un-coupled channels independently
channels_uncoupled, asymptotics_uncoupled = sys.uncoupled(Ecm, mu, k, eta)

# for solving as a single (block-diagonal) system
channels_coupled, asymptotics_coupled = sys.coupled(Ecm, mu, k, eta)

# the 2-level system is block diagonal consisting of 2 copies of the
# un-coupled scalar subsystems
params_2level = (10, 4, 0)
params_scalar = (10, 4)

In [9]:
Rm, Sm, xm = solver.solve(
    channels_coupled,
    asymptotics_coupled,
    local_interaction=potential_2level,
    local_args=params_2level,
)

ValueError: Input must be 1- or 2-d.